In [1]:
#%matplotlib inline
import os, sys
sys.path.append('/Users/hhsieh/anaconda3/envs/astroconda/lib/python3.6/site-packages')
sys.path.append('/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import ccdproc as cp
from ccdproc import CCDData, ImageFileCollection
import numpy as np
import math, glob, os, bz2, subprocess, sys
import cosmics_py3
import sqlite3
from sqlite3 import Error
import scipy.signal as signal
import scipy.ndimage as ndimage
import datetime


In [2]:
def create_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print('Directory {:s} successfully created.'.format(path))
    else:
        print('Directory {:s} already exists.'.format(path))
    return None

def decompress_file_bzip2(filename):
    cmd = ['bzip2','-d',filename]
    process = subprocess.call(cmd)
    return None

def compress_file_fpack(filename):
    fpack_command  = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    delete_command = '/bin/rm'
    fpack_cmd = [fpack_command,filename]
    process = subprocess.call(fpack_cmd)
    if process == 0:
        delete_cmd = [delete_command,filename]
        process = subprocess.call(delete_cmd)
    else:
        print('fpack command failed.')
    return None

def decompress_file_fpack(filename):
    funpack_command = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
    delete_command  = '/bin/rm'
    funpack_cmd = [funpack_command,filename]
    process = subprocess.call(funpack_cmd)
    if process == 0:
        delete_cmd = [delete_command,filename]
        process = subprocess.call(delete_cmd)
    else:
        print('funpack command failed.')
    return None

def decompress_directory_bzip2(file_path):
    print('>>> Starting decompression of bz2 files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for bz2_file in sorted(glob.glob('*.bz2')):
        decompress_file_bzip2(bz2_file)
    print('>>> Decompression of bz2 files complete.')
    return None

def compress_directory_fpack(file_path):
    print('>>> Starting fpack compression of FITS files in {:s}...'.format(file_path))
    #fpack_file = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    #delete_file = '/bin/rm'
    os.chdir(file_path)
    for fits_file in sorted(glob.glob('*.fits')):
        compress_file_fpack(fits_file)
    print('>>> fpack compression of FITS files complete.')
    return None
    
def decompress_directory_fpack(file_path):
    print('>>> Starting decompression of fz files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fz_file in sorted(glob.glob('*.fz')):
        decompress_file_fpack(fz_file)
    print('>>> Decompression of fz files complete.')
    return None


def get_date_id():
    # obtain YYYYMMDD date from filename of GMOS FITS files being processed
    # return: YYYYMMDD date id
    for raw_mef_file in sorted(glob.glob('*.fits')):
        fits_date = raw_mef_file[1:9]
    return fits_date

def create_stats_files(fits_date,imgtype):
    for extid in range(2,10):
        ext_stats = 'n{:s}.{:s}.{:02d}.stats'.format(fits_date,imgtype,extid+1)
        outputfile = open(ext_stats,'w')
        outputfile.write('# Extension {:02d}              NPIX        MEAN     STDDEV         MIN         MAX\n'.format(extid+1))
        outputfile.close()
    return None
        
def compile_stats_files(fits_date,imgtype):
    print('Starting image statistics output...')
    output_stats_filename = 'n{:s}.{:s}.stats'.format(fits_date,imgtype)
    output_stats_file = open(output_stats_filename,'w')
    cmd1 = '/bin/rm'
    for stats_filename in glob.glob('*.??.stats'):
        stats_file = open(stats_filename,'r')
        for line in stats_file:
            output_stats_file.write(line)
        stats_file.close()
        cmd = [cmd1,stats_filename]
        process = subprocess.call(cmd)
    output_stats_file.close()
    print('>>> Image statistics output complete.')
    return None

#def compile_stats_files(fits_date,outputfile_path):
#    print('Starting image statistics output...')
#    output_stats_filename = outputfile_path + 'n' + fits_date + '.stats'
#    output_stats_file = open(output_stats_filename,'w')
#    cmd1 = '/bin/rm'
#    for stats_filename in glob.glob('*.??.stats'):
#        stats_file = open(stats_filename,'r')
#        for line in stats_file:
#            output_stats_file.write(line)
#        stats_file.close()
#        cmd = [cmd1,stats_filename]
#        process = subprocess.call(cmd)
#    output_stats_file.close()
#    print('>>> Image statistics output complete.')
#    return None

def split_extensions(imgtype):
    # split 6-element GMOS multi-extension FITS files into individual elements
    # return: dimensions of extensions; also writes individual extension files to working directory
    for raw_mef_file in glob.glob('*.fits'):
        hdulist = fits.open(raw_mef_file)
        print('Splitting {:s}...'.format(raw_mef_file))
        fits_date = raw_mef_file[1:9]
        fits_id   = raw_mef_file[11:14]
        hdr1 = getheader(raw_mef_file,0)
        for extid in range(0,6):
            extension = hdulist[extid+1].data
            hdr2 = hdulist[extid+1].header
            radecsys = hdr2['RADECSYS']
            hdr2['RADESYSA'] = radecsys
            del hdr2['RADECSYS']
            imstat_npix = extension.size
            imstat_min = np.min(extension)
            imstat_max = np.max(extension)
            imstat_mean = np.mean(extension)
            imstat_stdev = np.std(extension)
            ext_filename = 'n' + fits_date + '.' + fits_id + '.{:02d}.fits'.format(extid+1)
            outputfilename = 'n{:s}.{:s}.{:02d}.stats'.format(fits_date,imgtype,extid+1)
            #outputfilename = 'n' + fits_date + '.{:02d}.stats'.format(extid+1)
            outputfile = open(outputfilename,'a')
            outputfile.write('{:s}   {:>8d}    {:>8.2f}    {:>7.2f}    {:>8.2f}    {:>8.2f}\n'.format(ext_filename,imstat_npix,imstat_mean,imstat_stdev,imstat_min,imstat_max))
            outputfile.close()
            dimensions = extension.shape
            dimension1 = dimensions[0]
            dimension2 = dimensions[1]
            hdr = hdr1 + hdr2
            fits.writeto(ext_filename,extension,hdr)
        hdulist.close()
    return dimension1, dimension2
    print('{:s} - >>> Multi-extension fits file splitting complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None
    
def remove_wrong_dimensions(dimension1,dimension2):
    # identify bias files with correct binning and remove others
    # return: none; retains bias files with correct binning and removes others
    for raw_mef_file in glob.glob('*.fits'):
        hdulist = fits.open(raw_mef_file)
        extension01 = hdulist[1].data
        bias_dimensions = extension01.shape
        bias_dimension1 = bias_dimensions[0]
        bias_dimension2 = bias_dimensions[1]
        hdulist.close()
        delete_file = '/bin/rm'
        if (bias_dimension1 != dimension1) or (bias_dimension2 != dimension2):
            delete_cmd = [delete_file,raw_mef_file]
            process = subprocess.call(delete_cmd)
    print('>>> Bias frame selection complete.')
    return None

def add_fits_header_line(fits_file,param_name,param_value,param_comment):
    # Add line to FITS header
    hdulist = fits.open(fits_file)
    data = hdulist[0].data
    hdr = hdulist[0].header
    hdr[param_name] = (param_value,param_comment)
    fits.writeto(fits_file,data,hdr,overwrite=True)
    return None

def overscan_and_trim_gmosn():
    # Do overscan correction and trim images in current directory
    print('{:s} - Starting overscan correction and trimming...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    for fits_file in glob.glob('*.???.??.fits'):
        ot_fits_file = fits_file[0:16] + '.ot.fits'
        fits_data = CCDData.read(fits_file,unit=u.adu)
        file_ext = fits_file[14:16]
        # Overscan correction
        if ((file_ext == '01') or (file_ext == '03')):
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[1:32,1:2304]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        else: # if file_ext == 4 or 5
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[513:544,1:2304]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        # Image trimming for Ext #s 1,3,4,5
        if ((file_ext == '01') or (file_ext == '03')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[33:543,3:2260]',add_keyword={'trimmed': True, 'calstat': 'OT'})
            ot_fits_data.write(ot_fits_file)
        elif ((file_ext == '04') or (file_ext == '05')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[2:512,3:2260]',add_keyword={'trimmed': True, 'calstat': 'OT'})
            ot_fits_data.write(ot_fits_file)
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)
    print('{:s} - >>> Overscan correction and trimming complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def bias_median_combine(dateid):
    print('{:s} - Starting median combination of bias frames...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    for extid in range(0,6):
        if((extid == 0) or (extid == 2) or (extid == 3) or (extid == 4)):
            bias_list = []
            ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
            output_filename = 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
            for fits_file in glob.glob(ext_file_format):
                fits_data = CCDData.read(fits_file)
                bias_list.append(fits_data)
                delete_file = '/bin/rm'
                delete_cmd = [delete_file,fits_file]
                process = subprocess.call(delete_cmd)
            master_bias = cp.combine(bias_list,method='median')
            master_bias.write(output_filename)
    print('{:s} - >>> Median combination of bias frames complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def bias_correct(dateid,path_raw_bias):
    print('{:s} - Starting bias subtraction...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    delete_cmd = '/bin/rm'
    for extid in range(0,6):
        if((extid == 0) or (extid == 2) or (extid == 3) or (extid == 4)):
            bias_filename = path_raw_bias + 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
            ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
            for fits_file in glob.glob(ext_file_format):
                fits_data = CCDData.read(fits_file)
                bias_data = CCDData.read(bias_filename)
                fits_date_imageid = fits_file[0:13]
                output_filename = fits_date_imageid + '.{:02d}.otz.fits'.format(extid+1)
                bias_corrected_data = cp.subtract_bias(fits_data,bias_data,add_keyword={'zerocorr':True,'calstat':'OTZ'})
                bias_corrected_data.write(output_filename)
                delete_file = [delete_cmd,fits_file]
                process = subprocess.call(delete_file)
    print('{:s} - >>> Bias subtraction complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def concatenate_gmos_amps():
    print('{:s} - >>> Starting adjacent amp area concatenation...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    delete_cmd = '/bin/rm'
    move_cmd = 'mv'
    
    for fits_file in glob.glob('*.01.otz.fits'):
        file_prefix = fits_file[0:13]
        output_filename = file_prefix + '.chip1.otz.fits'
        move_file = [move_cmd,fits_file,output_filename]
        process = subprocess.call(move_file)

    for fits_file in glob.glob('*.03.otz.fits'):
        file_prefix = fits_file[0:13]
        ext1_filename = file_prefix + '.04.otz.fits'
        ext2_filename = file_prefix + '.03.otz.fits'
        hdr = getheader(ext1_filename,0)
        hdulist1 = fits.open(ext1_filename)
        ext1_data = hdulist1[0].data
        hdulist2 = fits.open(ext2_filename)
        ext2_data = hdulist2[0].data
        hdulist1.close()
        hdulist2.close()
        chip1_data = np.hstack((ext1_data,ext2_data))
        output_filename = file_prefix + '.chip2.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        delete_file = [delete_cmd,ext1_filename]
        process = subprocess.call(delete_file)
        delete_file = [delete_cmd,ext2_filename]
        process = subprocess.call(delete_file)
        
    for fits_file in glob.glob('*.05.otz.fits'):
        file_prefix = fits_file[0:13]
        output_filename = file_prefix + '.chip3.otz.fits'
        move_file = [move_cmd,fits_file,output_filename]
        process = subprocess.call(move_file)
        
    print('{:s} - >>> Adjacent amp area concatenation complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def flatfield_correction(base_dir,data_dir,flatfield_chip1_file,flatfield_chip2_file,flatfield_chip3_file):
    print('{:s} - Starting flatfield correction...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    for fits_file in sorted(glob.glob('*.chip1.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(base_dir+data_dir+'/'+flatfield_chip1_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip1.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.01.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2','N/A','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3','N/A','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4','N/A','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip1_file,'Flatfield frame used')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    for fits_file in sorted(glob.glob('*.chip2.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(base_dir+data_dir+'/'+flatfield_chip2_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip2.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.03.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2',fits_date_imageid+'.bias.04.fits','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3','N/A','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4','N/A','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip2_file,'Flatfield frame used')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    for fits_file in sorted(glob.glob('*.chip3.otz.fits')):
        fits_data = CCDData.read(fits_file)
        flat_data = CCDData.read(base_dir+data_dir+'/'+flatfield_chip3_file,unit=u.adu)
        fits_date_imageid = fits_file[0:13]
        output_filename = fits_date_imageid + '.chip3.otzf.fits'
        flat_corrected_data = cp.flat_correct(fits_data,flat_data,add_keyword={'flatcorr': True, 'calstat': 'OTZF'})
        flat_corrected_data.write(output_filename)
        add_fits_header_line(output_filename,'BIAS_1',fits_date_imageid+'.bias.05.fits','1st bias frame used')
        add_fits_header_line(output_filename,'BIAS_2','N/A','2nd bias frame used')
        add_fits_header_line(output_filename,'BIAS_3','N/A','3rd bias frame used')
        add_fits_header_line(output_filename,'BIAS_4','N/A','4th bias frame used')
        add_fits_header_line(output_filename,'FLATUSED',flatfield_chip3_file,'Flatfield frame used')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    print('{:s} - >>> Flatfield correction complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def cosmicray_cleaning():
    print('{:s} - Starting cosmic ray correction...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    for fits_file in sorted(glob.glob('*.chip1.otzf.fits')):
        print('Cosmic ray cleaning for {:s}'.format(fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    for fits_file in sorted(glob.glob('*.chip2.otzf.fits')):
        print('Cosmic ray cleaning for {:s}'.format(fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)

    for fits_file in sorted(glob.glob('*.chip3.otzf.fits')):
        print('{:s} - Cosmic ray cleaning for {:s}'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),fits_file))
        file_prefix = fits_file[0:19]
        hdr = getheader(fits_file,0)
        fits_gain = hdr['gain']
        fits_readnoise = hdr['rdnoise']
        array,header = cosmics_py3.fromfits(fits_file,verbose=False)
        c = cosmics_py3.cosmicsimage(array,gain=fits_gain,readnoise=fits_readnoise,sigclip=5.0,sigfrac=0.3,objlim=5.0,verbose=False)
        c.run(maxiter=4)
        output_file = file_prefix + '.otzfc.fits'
        cosmics_py3.tofits(output_file,c.cleanarray,header,verbose=False)
        add_fits_header_line(output_file,'CRCORR',True,'Cosmic ray removal performed? (T/F)')
        add_fits_header_line(output_file,'CALSTAT','OTZFC','Image calibration status')
        add_fits_header_line(output_file,'CRR_GAIN',fits_gain,'CR removal parameter: gain')
        add_fits_header_line(output_file,'CRR_NOIS',fits_readnoise,'CR removal parameter: read noise')
        add_fits_header_line(output_file,'CRR_SIGC',5.0,'CR removal parameter: sigclip')
        add_fits_header_line(output_file,'CRR_SIGF',0.3,'CR removal parameter: sigfrac')
        add_fits_header_line(output_file,'CRR_OBJL',5.0,'CR removal parameter: objlim')
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)
        
    print('{:s} - >>> Cosmic ray correction complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def move_reduced_data(rawfitsdir,rawbiasdir,flatdir,path_stats):
    move_file = 'mv'
    os.chdir(rawfitsdir)
    for fits_file in sorted(glob.glob('*.otzfc.fits')):
        move_cmd = [move_file,fits_file,flatdir]
        process = subprocess.call(move_cmd)
    for stats_file in sorted(glob.glob('*.stats')):
        move_cmd = [move_file,stats_file,path_stats]
        process = subprocess.call(move_cmd)
    os.chdir(rawbiasdir)
    for fits_file in sorted(glob.glob('*.bias.??.fits')):
        move_cmd = [move_file,fits_file,flatdir]
        process = subprocess.call(move_cmd)
    for stats_file in sorted(glob.glob('*.stats')):
        move_cmd = [move_file,stats_file,path_stats]
        process = subprocess.call(move_cmd)
    print('{:s} - >>> Processed data moved to output directory.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

#def move_flatfield_images(base_dir,data_dir,path_procfits,flatfield_chip1,flatfield_chip2,flatfield_chip3):
#    move_file = 'mv'
#    os.chdir(base_dir + data_dir)
#    move_cmd = [move_file,flatfield_chip1,path_procfits]
#    process = subprocess.call(move_cmd)
#    move_cmd = [move_file,flatfield_chip2,path_procfits]
#    process = subprocess.call(move_cmd)
#    move_cmd = [move_file,flatfield_chip3,path_procfits]
#    process = subprocess.call(move_cmd)
#    return None

    
print('Done.')


Done.


In [3]:
def fix_gain_rdnoise_gmosn_e2v():
    # Fix gain and read noise values in FITS headers
    for fits_file in sorted(glob.glob('*.???.??.fits')):
        with fits.open(fits_file) as hdulist:
            data = hdulist[0].data
            hdr = hdulist[0].header
        hdr['GAIN'] = 2.27
        hdr['RDNOISE'] = 3.32
        fits.writeto(fits_file,data,hdr,overwrite=True)
    print('>>> Gain and read noise updated in image headers.')
    return None

def convert_to_long_target_desig(short_desig):
    long_desig = short_desig
    if short_desig == '133P': long_desig = '133P/1996 N2 (Elst-Pizarro 1)'
    if short_desig == '176P': long_desig = '176P/1999 RE70 (LINEAR 52)'
    if short_desig == '238P': long_desig = '238P/2005 U1 (Read 3)'
    if short_desig == '259P': long_desig = '259P/2008 R1 (Garradd 4)'
    if short_desig == '2006 VW139': long_desig = '288P/2006 VW139 (300163)'
    if short_desig == '313P': long_desig = '313P/2014 S4 (Gibbs 16)'
    if short_desig == '324P': long_desig = '324P/2010 R2 (La Sagra 4)'
    if short_desig == '358P': long_desig = '358P/2012 T1 (PanSTARRS 11)'
    return long_desig

def fix_gmos_header_info(instr_id,filter_id):
    # Fix header information for PDS submission
    for fits_file in sorted(glob.glob('*.otzfc.fits')):
        with fits.open(fits_file) as hdulist:
            #hdulist = fits.open(fits_file)
            data = hdulist[0].data
            hdr = hdulist[0].header
            
        try:
            # Change keywords for CDi_j parameters (conflict with other WCS keywords)
            if 'CD1_1' in hdr:
                cd1_1,cd1_1_comment = hdr['CD1_1'],hdr.comments['CD1_1']
                cd1_2,cd1_2_comment = hdr['CD1_2'],hdr.comments['CD1_2']
                cd2_1,cd2_1_comment = hdr['CD2_1'],hdr.comments['CD2_1']
                cd2_2,cd2_2_comment = hdr['CD2_2'],hdr.comments['CD2_2']
                hdr['x_CD1_1'] = (cd1_1,cd1_1_comment)
                hdr['x_CD1_2'] = (cd1_2,cd1_2_comment)
                hdr['x_CD2_1'] = (cd2_1,cd2_1_comment)
                hdr['x_CD2_2'] = (cd2_2,cd2_2_comment)
                del hdr['CD1_1']
                del hdr['CD1_2']
                del hdr['CD2_1']
                del hdr['CD2_2']
        except:
            print('Changing keywords for CDi_j parameters failed.')
        
        if 'EPOCH' in hdr:
            # Change keyword for EPOCH parameter
            target_epoch,target_epoch_comment = hdr['EPOCH'],hdr.comments['EPOCH']
            hdr['TRGEPOCH'] = (target_epoch,target_epoch_comment)
            del hdr['EPOCH']
        
        if 'RADVEL' in hdr:
            # Remove RADVEL parameter (not correctly populated)
            del hdr['RADVEL']
        
        # Change EXPTIME value to integer format
        exptime = int(round(hdr['EXPTIME']))
        hdr['EXPTIME'] = exptime
        
        # Add keyword for full object designation
        object_name = hdr['OBJECT']
        object_desig = convert_to_long_target_desig(object_name)
        hdr['OBJDESIG'] = object_desig
        
        # Add central filter wavelength keyword
        if instr_id == 'gmosn':
            if filter_id == 'g':   hdr['CENTWAVE'] = 475
            elif filter_id == 'r': hdr['CENTWAVE'] = 630
            elif filter_id == 'i': hdr['CENTWAVE'] = 780
        elif instr_id == 'gmoss':
            if filter_id == 'g':   hdr['CENTWAVE'] = 475
            elif filter_id == 'r': hdr['CENTWAVE'] = 630
            elif filter_id == 'i': hdr['CENTWAVE'] = 780
        
        # Remove checksums
        if 'CHECKSUM' in hdr: del hdr['CHECKSUM']
        if 'DATASUM'  in hdr: del hdr['DATASUM']
        
        fits.writeto(fits_file,data,hdr,overwrite=True,checksum=True)
            
        #with fits.open(fits_file,checksum=True) as hdu1:
        #    hdr = hdu1[0].header
        #    print(hdr['CHECKSUM'])
        #    print(hdr['DATASUM'])
        
    print('{:s} - >>> Header information fixed for PDS submission.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))

    return None

def find_flatfield_files(base_dir,data_dir,instr_id,filter_id):
    ff_chip1_filename,ff_chip2_filename,ff_chip3_filename = 'none','none','none'
    os.chdir(base_dir + data_dir)
    decompress_directory_fpack(base_dir + data_dir)
    flatfield_filename_pattern = 'n*.' + instr_id + '.twiskyflat.' + filter_id + '*.fits'
    if len(glob.glob(flatfield_filename_pattern)) != 3:
        print('Incorrect number of flatfield files found.')
    else:
        for flatfield_file in sorted(glob.glob(flatfield_filename_pattern)):
            if flatfield_file[-10:-5] == 'chip1': ff_chip1_filename = flatfield_file
            if flatfield_file[-10:-5] == 'chip2': ff_chip2_filename = flatfield_file
            if flatfield_file[-10:-5] == 'chip3': ff_chip3_filename = flatfield_file
    return ff_chip1_filename,ff_chip2_filename,ff_chip3_filename
                
print('Done.')


Done.


In [4]:
def process_images(base_dir,data_dir,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id):
    print('\n{:s} - Processing science data in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))

    flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename = find_flatfield_files(base_dir,data_dir,instr_id,filter_id)

    if flatfield_chip1_filename != 'none' and flatfield_chip2_filename != 'none' and flatfield_chip3_filename != 'none':
        if not os.path.isdir(path_procfits): os.mkdir(path_procfits)
        if not os.path.isdir(path_stats):    os.mkdir(path_stats)
        # Process raw science fits files
        print('\n{:s} - Decompressing and splitting data files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        os.chdir(path_rawfits)
        decompress_directory_bzip2(path_rawfits)       # decompress downloaded GMOS files
        decompress_directory_fpack(path_rawfits)       # decompress previously fpacked GMOS files
        date_id = get_date_id()
        create_stats_files(date_id,'scidata')    # create files to record image statistics
        dim1, dim2 = split_extensions('scidata') # split science MEFs into individual FITS files
        if instr_id == 'gmosn':
            fix_gain_rdnoise_gmosn_e2v()
        elif instr_id == 'gmoss':
            fix_gain_rdnoise_gmoss_e2v()
        compile_stats_files(date_id,'scidata')   # collect image stats of science files together and delete individual files

        # decompress raw bias fits files
        print('\n{:s} - Decompressing, filtering, and splitting bias files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
        os.chdir(path_rawbias)
        decompress_directory_bzip2(path_rawbias)       # decompress downloaded GMOS files
        decompress_directory_fpack(path_rawbias)       # decompress previously fpacked GMOS files
        remove_wrong_dimensions(dim1,dim2)       # find and retain bias frames matching data file binning
        create_stats_files(date_id,'bias')       # create files to record image statistics
        split_extensions('bias')                 # split bias MEFs into individual FITS files
        if instr_id == 'gmosn':
            fix_gain_rdnoise_gmosn_e2v()
        if instr_id == 'gmoss':
            fix_gain_rdnoise_gmoss_e2v()
        compile_stats_files(date_id,'bias')      # collect image stats of bias files together and delete individual files
    
        # perform overscan corrections and trim data for bias frames
        os.chdir(path_rawbias)
        if instr_id == 'gmosn':
            overscan_and_trim_gmosn()
        if instr_id == 'gmoss':
            overscan_and_trim_gmoss()
        bias_median_combine(date_id)

        # Process science frames
        os.chdir(path_rawfits)
        if instr_id == 'gmosn':
            overscan_and_trim_gmosn()                                           # Perform overscan correction and trim data
        if instr_id == 'gmoss':
            overscan_and_trim_gmoss()                                           # Perform overscan correction and trim data
        bias_correct(date_id,path_rawbias)                                      # Perform bias correction
        concatenate_gmos_amps()                                                 # Join science frames
        flatfield_correction(base_dir,data_dir,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)  # Do flatfield corrections
        cosmicray_cleaning()                                                    # Do cosmic ray cleaning
        fix_gmos_header_info(instr_id,filter_id)
    
        # Clean up data
        move_reduced_data(path_rawfits,path_rawbias,path_procfits,path_stats)
        #move_flatfield_images(base_dir,data_dir,path_procfits,flatfield_chip1_filename,flatfield_chip2_filename,flatfield_chip3_filename)
        compress_directory_fpack(base_dir+data_dir)
        compress_directory_fpack(path_rawfits)
        compress_directory_fpack(path_rawbias)
        compress_directory_fpack(path_procfits)
        print('{:s} - Processing science data in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
        
    else:
        print('{:s} - Appropriate flatfield images for {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),dir_rawfits))        

    return None


print('Done.')


Done.


In [5]:
#base_dir     = '/volumes/Fantom12a/BackupData/gemini/LLP_data/gemini_data.GN-2016B-LP-11/'
#base_dir     = '/volumes/Fantom12a/BackupData/gemini/data_queue_programs/gemini_data.GN-2013A-Q-102/'
#base_dir     = '/volumes/Fantom12a/BackupData/gemini/data_queue_programs/gemini_data.GN-2016A-Q-88/'
base_dir     = '/volumes/Fantom12a/BackupData/Data2012/'

dir_rawbias  = 'rawfits_bias'
dir_procfits = 'flatfits_pyt'
dir_stats    = 'stats'

os.chdir(base_dir)
for data_dir in sorted(glob.glob('ut*_gemini?')):
    
    if data_dir[-1:] == 'N':   instr_id = 'gmosn'
    elif data_dir[-1:] == 'S': instr_id = 'gmoss'
    else: instr_id = 'not recognized'

    if instr_id == 'gmosn' or instr_id == 'gmoss':
        os.chdir(base_dir + data_dir)
        if len(glob.glob('rawfits_?')) > 0:
            for dir_rawfits in sorted(glob.glob('rawfits_?')):
                filter_id = dir_rawfits[-1:]    
                path_rawfits  = base_dir + data_dir + '/' + dir_rawfits  + '/'
                path_rawbias  = base_dir + data_dir + '/' + dir_rawbias  + '/'
                path_procfits = base_dir + data_dir + '/' + dir_procfits + '/'
                path_stats    = base_dir + data_dir + '/' + dir_stats    + '/'

                if os.path.exists(path_rawfits):
                    if not os.path.exists(path_procfits):
                        if os.path.exists(path_rawbias):
                            if len(os.listdir(path_rawbias)) > 0:
                                process_images(base_dir,data_dir,path_rawfits,path_rawbias,path_procfits,path_stats,instr_id,filter_id)
                            else:
                                print('{:s} - Bias directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                        else:
                            print('{:s} - Bias directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                    else:
                        print('{:s} - Science data in {:s} already processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
                else:
                    print('{:s} - {:s}-band science data in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filter_id,data_dir))
        else:
            print('{:s} - Science data in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))
    else:
        print('{:s} - Directory name format not recognized for {:s}.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),data_dir))

print('Done.')



2021-03-02 11:37:46 - Processing science data in ut20120125_geminiN...
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/Data2012/ut20120125_geminiN...
>>> Decompression of fz files complete.

2021-03-02 11:37:49 - Decompressing and splitting data files...
>>> Starting decompression of bz2 files in /volumes/Fantom12a/BackupData/Data2012/ut20120125_geminiN/rawfits_r/...
>>> Decompression of bz2 files complete.
>>> Starting decompression of fz files in /volumes/Fantom12a/BackupData/Data2012/ut20120125_geminiN/rawfits_r/...
>>> Decompression of fz files complete.
Splitting N20120125S0381.fits...
Splitting N20120125S0382.fits...
Splitting N20120125S0383.fits...
Splitting N20120125S0384.fits...
Splitting N20120125S0385.fits...
Splitting N20120125S0386.fits...
Splitting N20120125S0387.fits...
Splitting N20120125S0388.fits...
Splitting N20120125S0389.fits...
>>> Gain and read noise updated in image headers.
Starting image statistics output...
>>> Image statistics output

2021-03-02 11:40:48 - >>> Overscan correction and trimming complete.
2021-03-02 11:40:48 - Starting median combination of bias frames...


Invalid parameter values: MJD-OBS and DATE-OBS are inconsistent'. [astropy.wcs.wcs]
Invalid parameter values: MJD-OBS and DATE-OBS are inconsistent'. [astropy.wcs.wcs]
Invalid parameter values: MJD-OBS and DATE-OBS are inconsistent'. [astropy.wcs.wcs]
Invalid parameter values: MJD-OBS and DATE-OBS are inconsistent'. [astropy.wcs.wcs]


2021-03-02 11:41:54 - >>> Median combination of bias frames complete.
2021-03-02 11:41:54 - Starting overscan correction and trimming...


2021-03-02 11:42:45 - >>> Overscan correction and trimming complete.
2021-03-02 11:42:45 - Starting bias subtraction...


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
2021-03-02 20:03:55 - >>> Flatfield correction complete.
2021-03-02 20:03:55 - Starting cosmic ray correction...
Cosmic ray cleaning for n20121214.338.chip1.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 0 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
1132 cosmic pixels (1132 new)
Iteration 2
330 cosmic pixels (254 new)
Iteration 3
129 cosmic pixels (95 new)
Iteration 4
62 cosmic pixels (24 new)
Cosmic ray cleaning for n20121214.339.chip1.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 0 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
1174 cosmic pixels (1174 new)
Iteration 2
303 cosmic pixels (239 new)
Iteration 3
125 cosmic pixels (80 new)
Iteration 4
81 cosmic pixels (44 new)
Cosmic ray cleaning for

1173 cosmic pixels (1173 new)
Iteration 2
297 cosmic pixels (244 new)
Iteration 3
110 cosmic pixels (76 new)
Iteration 4
37 cosmic pixels (19 new)
Cosmic ray cleaning for n20121214.360.chip1.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 0 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
1255 cosmic pixels (1255 new)
Iteration 2
301 cosmic pixels (258 new)
Iteration 3
100 cosmic pixels (78 new)
Iteration 4
46 cosmic pixels (26 new)
Cosmic ray cleaning for n20121214.361.chip1.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 0 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
1129 cosmic pixels (1129 new)
Iteration 2
245 cosmic pixels (200 new)
Iteration 3
98 cosmic pixels (57 new)
Iteration 4
40 cosmic pixels (18 new)
Cosmic ray cleaning for n20121214.362.chip1.otzf.fits
Detecting saturated stars ...
Building mask of saturat

417 cosmic pixels (417 new)
Iteration 2
32 cosmic pixels (32 new)
Iteration 3
0 cosmic pixels (0 new)
Cosmic ray cleaning for n20121214.348.chip2.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 4 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
275 cosmic pixels (275 new)
Iteration 2
25 cosmic pixels (25 new)
Iteration 3
2 cosmic pixels (2 new)
Iteration 4
0 cosmic pixels (0 new)
Cosmic ray cleaning for n20121214.349.chip2.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 3 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
295 cosmic pixels (295 new)
Iteration 2
16 cosmic pixels (16 new)
Iteration 3
0 cosmic pixels (0 new)
Cosmic ray cleaning for n20121214.350.chip2.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 4 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterati

0 cosmic pixels (0 new)
Cosmic ray cleaning for n20121214.372.chip2.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 4 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
467 cosmic pixels (467 new)
Iteration 2
43 cosmic pixels (43 new)
Iteration 3
0 cosmic pixels (0 new)
2021-03-02 20:23:34 - Cosmic ray cleaning for n20121214.338.chip3.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 5 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
1134 cosmic pixels (1134 new)
Iteration 2
316 cosmic pixels (245 new)
Iteration 3
109 cosmic pixels (76 new)
Iteration 4
58 cosmic pixels (27 new)
2021-03-02 20:23:46 - Cosmic ray cleaning for n20121214.339.chip3.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 5 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
1210 

50 cosmic pixels (14 new)
2021-03-02 20:27:49 - Cosmic ray cleaning for n20121214.358.chip3.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 5 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
1101 cosmic pixels (1101 new)
Iteration 2
347 cosmic pixels (267 new)
Iteration 3
113 cosmic pixels (60 new)
Iteration 4
59 cosmic pixels (13 new)
2021-03-02 20:28:02 - Cosmic ray cleaning for n20121214.359.chip3.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 4 saturated stars.
Mask of saturated stars done
Starting 4 L.A.Cosmic iterations ...
Iteration 1
1108 cosmic pixels (1108 new)
Iteration 2
378 cosmic pixels (290 new)
Iteration 3
137 cosmic pixels (82 new)
Iteration 4
61 cosmic pixels (25 new)
2021-03-02 20:28:15 - Cosmic ray cleaning for n20121214.360.chip3.otzf.fits
Detecting saturated stars ...
Building mask of saturated stars ...
We have 5 saturated stars.
Mask of satura